In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, RobustScaler
from sklearn.ensemble import RandomForestClassifier


df_model = pd.read_csv('loan_project/SBAnational.csv')

/tmp/ipykernel_20264/4132040102.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/tmp/ipykernel_20264/4132040102.py:15: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_model = pd.read_csv('loan_project/SBAnational.csv')


In [2]:
df_model = df_model.drop(['Name','City','Bank','CreateJob','ApprovalDate','RetainedJob','ChgOffDate', 'DisbursementDate','DisbursementGross','BalanceGross','ChgOffPrinGr','SBA_Appv'],axis=1)
df_model

,LoanNr_ChkDgt,State,Zip,BankState,NAICS,ApprovalFY,Term,NoEmp,NewExist,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status,GrAppv
0,1000014003,IN,47711,OH,451120,1997,84,4,2.0,1,0,N,Y,P I F,"$60,000.00"
1,1000024006,IN,46526,IN,722410,1997,60,2,2.0,1,0,N,Y,P I F,"$40,000.00"
2,1000034009,IN,47401,IN,621210,1997,180,7,1.0,1,0,N,N,P I F,"$287,000.00"
3,1000044001,OK,74012,OK,0,1997,60,2,1.0,1,0,N,Y,P I F,"$35,000.00"
4,1000054004,FL,32801,FL,0,1997,240,14,1.0,1,0,N,N,P I F,"$229,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,9995573004,OH,43221,IL,451120,1997,60,6,1.0,1,0,0,N,P I F,"$70,000.00"
899160,9995603000,OH,43221,IL,451130,1997,60,6,1.0,1,0,Y,N,P I F,"$85,000.00"
899161,9995613003,CA,93455,CA,332321,1997,108,26,1.0,1,0,N,N,P I F,"$300,000.00"
899162,9995973006,HI,96830,HI,0,1997,60,6,1.0,1,0,N,Y,CHGOFF,"$75,000.00"


In [3]:
df_model.isnull().sum()

LoanNr_ChkDgt       0
State              14
Zip                 0
BankState        1566
NAICS               0
ApprovalFY          0
Term                0
NoEmp               0
NewExist          136
FranchiseCode       0
UrbanRural          0
RevLineCr        4528
LowDoc           2582
MIS_Status       1997
GrAppv              0
dtype: int64

In [4]:
df_model['ApprovalFY'] = pd.to_numeric(df_model['ApprovalFY'], errors='coerce').astype('Int64')
df_model['GrAppv'] = pd.to_numeric(df_model['GrAppv'].str.replace('[\$,]', '', regex=True))



df_model['RevLineCr'] = df_model['RevLineCr'].replace(('0','T','`',',','1','C','3','2','R','7','A','5','.','4','-','Q'), np.nan)

condition = (df_model['GrAppv'] > 150000)
df_model.loc[condition, 'LowDoc'] = 'N'
df_model.loc[~condition, 'LowDoc'] = 'Y'

imputer = SimpleImputer(strategy='most_frequent')
df_model[['RevLineCr','NewExist', 'LowDoc']] = imputer.fit_transform(df_model[['RevLineCr','NewExist', 'LowDoc']])

imputer = SimpleImputer(strategy='most_frequent')
df_model['ApprovalFY'] = imputer.fit_transform(df_model['ApprovalFY'])
df_model

ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

In [ ]:
df_model.isnull().sum()

LoanNr_ChkDgt    0
State            0
Zip              0
BankState        0
NAICS            0
ApprovalFY       0
Term             0
NoEmp            0
NewExist         0
FranchiseCode    0
UrbanRural       0
RevLineCr        0
LowDoc           0
MIS_Status       0
GrAppv           0
dtype: int64

In [ ]:
X = df_model.drop('MIS_Status', axis=1)
y = df_model.MIS_Status


X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=y)


num_col = list(X.select_dtypes(include=[float,int]).columns)
cat_col = list(X.select_dtypes(include=[object]).columns)

onehotscale_pipeline = make_pipeline(OneHotEncoder(), RobustScaler(with_centering=False))
scale_pipeline = make_pipeline(RobustScaler(with_centering=False))

preprocessing = ColumnTransformer(
    transformers=[
        ('categorical', onehotscale_pipeline, cat_col),
        ('numerical', scale_pipeline, num_col)]
)

my_final_pipeline = make_pipeline(preprocessing)
my_final_pipeline.fit(X_train)

feature_names = my_final_pipeline.get_feature_names_out(X.columns)

model = make_pipeline(
    my_final_pipeline,
    RandomForestClassifier(random_state=42,max_depth=3)
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

classes = model.classes_
print(classes)

report = classification_report(y_test, y_pred)

print(report)


print("Confusion Matrix:")
display(confusion_matrix(y_test, y_pred))

# faire une feature importance

['CHGOFF' 'P I F']


/home/utilisateur/Documents/Projets/US_SBA/MLProject/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/utilisateur/Documents/Projets/US_SBA/MLProject/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/utilisateur/Documents/Projets/US_SBA/MLProject/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

              precision    recall  f1-score   support

      CHGOFF       0.00      0.00      0.00     23623
       P I F       0.82      1.00      0.90    110724

    accuracy                           0.82    134347
   macro avg       0.41      0.50      0.45    134347
weighted avg       0.68      0.82      0.74    134347

Confusion Matrix:
[[     0  23623]
 [     0 110724]]
